<a href="https://colab.research.google.com/github/LDY681/CITS4404_project/blob/master/CITS4404.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CITS4404 Firefly Algorithm Trading Bot
#### by: Davinh Dang (22717235), Dayu Liu (24188516), Ethan Young (23450844), Flavian Jerotich (24001784), Kushan Jayasekera (24205163), Xin Wang (24201533)

# README
This folder contains this `ipynb` file to run bot training and trading, dataset is imported from kaggle.

# 1. Dataset Processing and Filter Implementation
This step involves processing kaggle dataset. Dates and Closing prices are extracted with their corresponding values, ready for use for further processing for the analysis.
### Get required Python Modules

In [ ]:
# TODO Required modules
!pip install torch
!pip install numpy
!pip install pandas
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 802.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Import libraries
# Essentials
import numpy as np

# Data Processing
import pandas as pd
import kagglehub

# FIXME Modeling
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import time


### Data Processing

In [31]:
# Download dataset from Kaggle
path = kagglehub.dataset_download("prasoonkottarathil/btcinusd")
print("Path to dataset files:", path)

fileName = "BTC-Daily.csv"  #! Other candidates include BTC-{2017-2021}min.csv, BTC-Hourly.csv (Which are hourly data and may not be suitable)

def preprocess_data(path, fileName):
    file_path = f"{path}/{fileName}"
    data = pd.read_csv(file_path)

    dates = data['date'].tolist()
    prices = data['close'].tolist()

    df = pd.DataFrame({'date': dates, 'price': prices})
    df['date'] = pd.to_datetime(df['date'])

    test_df = df[~df['date'].dt.year.isin([2020, 2021, 2022])]
    df = df.sort_values('date', ascending=True).reset_index(drop=True)
    truncated_df = test_df.sort_values('date', ascending=True).reset_index(drop=True)

    return [df, truncated_df]

[train_df, test_df] = preprocess_data(path, fileName)

# Variables for model
dates = train_df['date']
prices = train_df['price']
test_dates = test_df['date']
test_prices = test_df['price']

print("Dates:", dates[:5])
print("Prices:", prices[:5])
print("Test Dates:", test_dates[-5:])
print("Test Prices:", test_prices[-5:])

Path to dataset files: /kaggle/input/btcinusd
Dates: 0   2014-11-28
1   2014-11-29
2   2014-11-30
3   2014-12-01
4   2014-12-02
Name: date, dtype: datetime64[ns]
Prices: 0    376.28
1    376.72
2    373.34
3    378.39
4    379.25
Name: price, dtype: float64
Test Dates: 1855   2019-12-27
1856   2019-12-28
1857   2019-12-29
1858   2019-12-30
1859   2019-12-31
Name: date, dtype: datetime64[ns]
Test Prices: 1855    7246.00
1856    7296.24
1857    7385.54
1858    7220.24
1859    7168.36
Name: price, dtype: float64


### SMA Filter Implementation

In [ ]:
def pad(P, N):
    flip = np.flip(P[1:N]) # TODO The project instruction have a negate sign before the flip function, could be a typo
    return np.append(flip, P)

def sma_filter(P, N):
    # pad the series
    P_pad = pad(P, N)
    # Roll the window and calculate the weighted average
    sma_full = pd.Series(P_pad).rolling(window=N).mean().to_numpy()
    return sma_full[N-1:]

#! To generate an N-day SMA we can simply call
sma10 = sma_filter(prices, 10)
sma20 = sma_filter(prices, 20)
print("sma10:", sma10[20:25])

sma10: [42625.384 43034.841 43366.134 43218.01  42902.534]


# 2. Model Implementation (TODO)


### Model's Hyperparameters (TODO)

In [ ]:
# Variable Initialization
lb = 10 # Short-term moving average window size
ub = 40 # Long-term moving average window size

### Model Constructor

In [ ]:
# FIXME from https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/fireflyalgorithm/fireflyalgorithm.py
import numpy as np
from numpy.random import default_rng


class FireflyAlgorithm:
    def __init__(self, pop_size=20, alpha=1.0, betamin=1.0, gamma=0.01, seed=None):
        self.pop_size = pop_size
        self.alpha = alpha
        self.betamin = betamin
        self.gamma = gamma
        self.rng = default_rng(seed)

    def run(self, function, dim, lb, ub, max_evals):
        fireflies = self.rng.uniform(lb, ub, (self.pop_size, dim))
        intensity = np.apply_along_axis(function, 1, fireflies)
        best = np.min(intensity)

        evaluations = self.pop_size
        new_alpha = self.alpha
        search_range = ub - lb

        while evaluations <= max_evals:
            new_alpha *= 0.97
            for i in range(self.pop_size):
                for j in range(self.pop_size):
                    if intensity[i] >= intensity[j]:
                        r = np.sum(np.square(fireflies[i] - fireflies[j]), axis=-1)
                        beta = self.betamin * np.exp(-self.gamma * r)
                        steps = new_alpha * (self.rng.random(dim) - 0.5) * search_range
                        fireflies[i] += beta * (fireflies[j] - fireflies[i]) + steps
                        fireflies[i] = np.clip(fireflies[i], lb, ub)
                        intensity[i] = function(fireflies[i])
                        evaluations += 1
                        best = min(intensity[i], best)
        return best, fireflies[np.argmin(intensity)] # FIXME return not just the score but also best parameters

### Model Training (TODO)

In [ ]:
# FIXME Example usage using sphere as fitness from https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/fireflyalgorithm/problems.py and https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/examples/run.py
def sphere(x):
    return np.sum(x**2)
FA = FireflyAlgorithm()
best = FA.run(function=sphere, dim=10, lb=lb, ub=ub, max_evals=10000)
print(best)

# FIXME from ChatGPT on how to apply our own fitness with SMA
def firefly_fitness_wrapper(prices):
    def wrapped_firefly_fitness(X):
        sma1 = int(round(X[0]))
        sma2 = int(round(X[1]))
        if sma1 >= sma2:
            return 1e9  #! HIGH value = BAD fitness (we minimize)
        returns = backtest_strategy(sma1, sma2, prices)
        score = -compute_sharpe_ratio(returns)  # We minimize, so negate Sharpe ratio
        return score
    return wrapped_firefly_fitness

def backtest_strategy(SMA1, SMA2, prices):
    sma1 = prices.rolling(window=int(SMA1)).mean()
    sma2 = prices.rolling(window=int(SMA2)).mean()

    signal = np.where(sma1 > sma2, 1, 0)
    returns = prices.pct_change().shift(-1)  # Next-day return
    strategy_returns = signal[:-1] * returns[1:]
    return strategy_returns

def compute_sharpe_ratio(returns):
    mean_ret = np.mean(returns)
    std_ret = np.std(returns)
    if std_ret == 0:
        return 0
    return mean_ret / std_ret

firefly = FireflyAlgorithm(pop_size=25, alpha=0.5, betamin=0.2, gamma=1.0, seed=42)
fitness_fn = firefly_fitness_wrapper(prices)
best_score, best_params = firefly.run(fitness_fn, dim=2, lb=np.array([5, 10]), ub=np.array([30, 60]), max_evals=1000)
print(f"Best Sharpe Ratio: {best_score}")
print(f"Best Parameters: SMA1={best_params[0]}, SMA2={best_params[1]}")

(2734.820325344761, array([11.85363463, 24.05266677, 14.55685221, 31.68508371, 10.        ,
       12.68994667, 18.47098485, 14.66098479, 16.59621893, 31.31314818]))
Best Sharpe Ratio: -0.009866822220598415
Best Parameters: SMA1=30.0, SMA2=31.14481295921547


# 3. Trading strategy (TODO)

In [ ]:
def simulate_trading(prices, SMA1, SMA2, initial_capital=1000, fee_rate=0.03, printLogs=False):
    if SMA1 >= SMA2:
        print("Warning: Short-term SMA window (SMA1) should be less than long-term SMA window (SMA2).")
        return -1

    # --- 1. Calculate SMAs ---
    sma_short = sma_filter(prices, SMA1)
    sma_long = sma_filter(prices, SMA2)

    # --- 2. Generate Crossover Signals ---
    # Create a position state: 1 if sma_short > sma_long, -1 otherwise
    # Use -1 for below/equal to simplify crossover detection
    position_state = pd.Series(np.where(sma_short > sma_long, 1, -1), index=prices.index)

    # Detect crossovers:
    # Buy signal (1): state changes from -1 (below/equal) to 1 (above)
    # Sell signal (-1): state changes from 1 (above) to -1 (below/equal)
    # Use .diff() to find changes. A change from -1 to 1 is diff = 2. A change from 1 to -1 is diff = -2.
    crossover = position_state.diff()
    buy_signals = pd.Series(np.where(crossover == 2, 1, 0), index=prices.index)
    sell_signals = pd.Series(np.where(crossover == -2, -1, 0), index=prices.index)

    capital = float(initial_capital)
    shares = 0.0
    holding_bitcoin = False

    start_index = max(SMA1, SMA2) # First day both SMAs are potentially valid (depends on sma_filter implementation)
                                    # Crossover signal needs one more day due to diff()
                                    # Ensure loop starts where crossover signal is not NaN

    # --- 4. Simulation Loop ---
    # Iterate through days where signals *could* be generated
    for i in range(start_index, len(prices)):
        price_today = prices.iloc[i]
        buy_signal_today = buy_signals.iloc[i]
        sell_signal_today = sell_signals.iloc[i]

        # Check for invalid price
        if pd.isna(price_today) or price_today <= 0:
            continue # Skip day if price is invalid

        # If Buy signal generated today AND currently holding cash
        if buy_signal_today == 1 and not holding_bitcoin:
            cash_to_spend = capital * (1 - fee_rate) # fee
            shares = cash_to_spend / price_today
            capital = 0.0
            holding_bitcoin = True
            if printLogs: print(f"Day {prices.index[i]}\nBUYING @ {price_today:.2f}\nShares - USD: {shares:.6f}\n") #print buy

        # If Sell signal generated today AND currently holding Bitcoin
        elif sell_signal_today == -1 and holding_bitcoin:
            cash_received = shares * price_today
            capital = cash_received * (1 - fee_rate) # fee
            shares = 0.0
            holding_bitcoin = False
            if printLogs: print(f"Day {prices.index[i]}\nSELL @ {price_today:.2f}\nCapital - Bitcoin: {capital:.2f}\n") # Optional logging

    # At the end of the sequence, sell remaining bitcoin (if any) at the final price to see how much was earned
    if holding_bitcoin:
        final_price = prices.iloc[-1]
        if pd.notna(final_price) and final_price > 0:
             cash_received = shares * final_price
             capital = cash_received * (1 - fee_rate) # Fee
             shares = 0.0
             holding_bitcoin = False
             if printLogs: print(f"End Liquidation\nSELL @ {final_price:.2f}\n")
             print(f"Final capital for SMA({SMA1}, {SMA2}): Final Capital - USD: ${capital:.2f}\n")
    else:
        print(f"Final capital for SMA({SMA1}, {SMA2}): Final Capital - USD: ${capital:.2f}\n")

In [ ]:
# Assume you already got these from Firefly output:
best_sma1, best_sma2 = 10, 20

simulate_trading(prices, best_sma1, best_sma2)

Final capital for SMA(10, 100): Final Capital - USD: $217.33

